
### LangChain Agents + MCP + Semantic Kernel

This practical notebook teaches you how to build **agentic AI** for **Banking & Finance** using the latest:
- **LangChain (v0.3+) Agents** — modern tool‑calling agents
- **Model Context Protocol (MCP)** — bring external tools into your agent
- **Semantic Kernel (SK)** — planners & orchestration with GPT‑class models (configure **GPT‑5** if available)

> **Runs in Jupyter.** Cells are defensive: LLM sections gracefully skip if your API key isn't set.  
> **Model note:** Set `OPENAI_MODEL="gpt-5"` if your account has access; otherwise use a supported model like `gpt-4o` / `gpt-4o-mini`.



## 0) Environment & Versions

Uncomment the `pip install` cell if you need to install/upgrade packages.


In [1]:

# If needed, uncomment to install:
# %pip install -U langchain langchain-core langchain-openai #                langgraph langchain-mcp-adapters mcp #                semantic-kernel python-dotenv

import os, sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

# Optionally load secrets from a local .env
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Loaded .env")
except Exception:
    print("dotenv not used (ok)")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")  # set to "gpt-5" if you have access
print("OPENAI_API_KEY set:", bool(OPENAI_API_KEY))
print("OPENAI_MODEL:", OPENAI_MODEL)


Python: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
Platform: macOS-26.0.1-arm64-arm-64bit
Loaded .env
OPENAI_API_KEY set: True
OPENAI_MODEL: gpt-4o-mini



## 1) LangChain Agents (v0.3) — Fundamentals

We will build a **tool‑calling agent** with two BFSI‑flavoured tools:
- `fx_rate(tool)`: returns a mocked FX rate (e.g., USD→INR).
- `calc(tool)`: safely computes `a (+|-|*|/) b`.


In [2]:

from typing import TypedDict, Optional
import ast, operator as op

OPS = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv}

def safe_eval_expr(expr: str) -> float:
    node = ast.parse(expr, mode="eval").body
    def _eval(n):
        if isinstance(n, ast.Num):  # type: ignore[attr-defined]
            return float(n.n)
        if isinstance(n, ast.BinOp) and type(n.op) in OPS:
            return OPS[type(n.op)](_eval(n.left), _eval(n.right))
        raise ValueError("Only + - * / with numbers supported.")
    return _eval(node)

def fx_rate(pair: str = "USD/INR") -> str:
    base, quote = pair.split("/")
    table = {("USD","INR"): 84.15, ("EUR","INR"): 92.50, ("GBP","INR"): 107.80}
    val = table.get((base.upper(), quote.upper()))
    return f"{pair} ≈ {val} (mock)" if val else f"No mock rate for {pair}"

def calc(expr: str) -> str:
    try:
        return f"{safe_eval_expr(expr):g}"
    except Exception as e:
        return f"calc error: {e}"

print("Tools ready: fx_rate(), calc()")


Tools ready: fx_rate(), calc()


In [3]:

from langchain_core.tools import tool

@tool
def fx_rate_tool(pair: str = "USD/INR") -> str:
    """Return a mock FX rate for a currency pair like 'USD/INR'."""
    return fx_rate(pair)

@tool
def calc_tool(expr: str) -> str:
    """Compute a basic arithmetic expression like '12 + 5' or '100/4'."""
    return calc(expr)

tools = [fx_rate_tool, calc_tool]
[name for name in [t.name for t in tools]]


['fx_rate_tool', 'calc_tool']


### Build the agent (tool‑calling) and run a few tasks


In [4]:

USE_LLM = bool(OPENAI_API_KEY)
print("LLM enabled:", USE_LLM)

if USE_LLM:
    from langchain_openai import ChatOpenAI
    from langchain.agents import create_tool_calling_agent, AgentExecutor
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain.tools.render import render_text_description

    llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a banking assistant. You can use tools.\nTools:\n{tools}\n\nTool names: {tool_names}\nWhen done, reply ONLY as: Final Answer: <concise result>"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ])
    prompt = prompt.partial(
        tools=render_text_description(tools),
        tool_names=", ".join(t.name for t in tools),
    )

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

    import nest_asyncio, asyncio
    nest_asyncio.apply()

    async def run_examples():
        res1 = await executor.ainvoke({"input": "What's the USD/INR FX rate?" , "agent_scratchpad": []})
        res2 = await executor.ainvoke({"input": "Calculate 125 * 8", "agent_scratchpad": []})
        return res1, res2

    r1, r2 = await run_examples()
    print(r1.get("output",""))
    print(r2.get("output",""))
else:
    print("⚠️ Set OPENAI_API_KEY to run the agent demo.")


LLM enabled: True


> Entering new AgentExecutor chain...

Invoking: `fx_rate_tool` with `{'pair': 'USD/INR'}`


USD/INR ≈ 84.15 (mock)Final Answer: 84.15

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `calc_tool` with `{'expr': '125 * 8'}`


1000Final Answer: 1000

> Finished chain.
Final Answer: 84.15
Final Answer: 1000



## 2) MCP — Model Context Protocol (external tools)

Create a tiny echo server and connect via **stdio**.


In [5]:
from pathlib import Path
server_code = '''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Echo")

@mcp.tool()
def reverse(text: str) -> str:
    "Return the reversed string."
    return text[::-1]

@mcp.tool()
def upper(text: str) -> str:
    "Return the string in uppercase."
    return text.upper()

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''
Path("echo_server.py").write_text(server_code, encoding="utf-8")
print("Wrote echo_server.py — run it in another terminal:  python echo_server.py")


Wrote echo_server.py — run it in another terminal:  python echo_server.py


In [6]:
if USE_LLM:
    from langchain_mcp_adapters.client import MultiServerMCPClient
    from langchain.agents import create_tool_calling_agent, AgentExecutor
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain.tools.render import render_text_description
    from pathlib import Path
    from langchain_openai import ChatOpenAI

    client = MultiServerMCPClient({
        "echo": {
            "transport": "stdio",
            "command": "python",
            "args": [str(Path("echo_server.py").resolve())],
        }
    })
    tools_mcp = await client.get_tools()

    prompt2 = ChatPromptTemplate.from_messages([
        ("system",
         "You can use MCP tools. Tools:\n{tools}\n\nTool names: {tool_names}\nWhen done, reply ONLY as: Final Answer: <result>"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ])
    prompt2 = prompt2.partial(
        tools=render_text_description(tools_mcp),
        tool_names=", ".join(t.name for t in tools_mcp),
    )

    llm2 = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
    agent2 = create_tool_calling_agent(llm=llm2, tools=tools_mcp, prompt=prompt2)
    exec2 = AgentExecutor(agent=agent2, tools=tools_mcp, verbose=True, handle_parsing_errors=True)

    resA = await exec2.ainvoke({"input": "Reverse 'LangChain MCP'", "agent_scratchpad": []})
    resB = await exec2.ainvoke({"input": "Make uppercase: banking ai", "agent_scratchpad": []})
    print(resA.get("output",""))
    print(resB.get("output",""))
else:
    print("⚠️ Set OPENAI_API_KEY and run echo_server.py in another terminal to try MCP demo.")




> Entering new AgentExecutor chain...

Invoking: `reverse` with `{'text': 'LangChain MCP'}`


PCM niahCgnaL
Invoking: `upper` with `{'text': 'PCM niahCgnaL'}`


PCM NIAHCGNALFinal Answer: PCM NIAHCGNAL

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `upper` with `{'text': 'banking ai'}`


BANKING AIFinal Answer: BANKING AI

> Finished chain.
Final Answer: PCM NIAHCGNAL
Final Answer: BANKING AI



## 3) Semantic Kernel (SK) — planners with GPT‑class models (GPT‑5 if available)

Minimal pattern (adjust imports per your SK version).


In [7]:
try:
    import semantic_kernel as sk
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    print("Semantic Kernel imported.")

    kernel = sk.Kernel()
    api_key = OPENAI_API_KEY or "missing"
    service = OpenAIChatCompletion(OPENAI_MODEL, api_key=api_key)
    kernel.add_text_completion_service("openai", service)

    goal = "Summarize RBI policy changes impacting home loans in 3 bullet points."
    prompt = f"Plan steps to achieve this goal, then produce the final 3-bullet summary.\nGoal: {goal}"
    result = await kernel.complete_text_async(prompt)
    print("SK Result:\n", result)
except Exception as e:
    print("⚠️ SK demo skipped or adjust imports for your SK version:", e)


Semantic Kernel imported.
⚠️ SK demo skipped or adjust imports for your SK version: 'Kernel' object has no attribute 'add_text_completion_service'



## 4) Router: plan tasks → SK, operational tasks → Agent


In [8]:

from dataclasses import dataclass

@dataclass
class Request:
    text: str

async def handle_request(req: Request):
    txt = req.text.strip()
    if txt.lower().startswith("plan:"):
        try:
            import semantic_kernel as sk  # ensure available
            return "Planner route:\n" + (await kernel.complete_text_async(txt))
        except Exception as e:
            return f"Planner route unavailable: {e}"
    else:
        if not USE_LLM:
            return "Agent route blocked: set OPENAI_API_KEY."
        out = await executor.ainvoke({"input": txt, "agent_scratchpad": []})
        return out.get("output","")

print(await handle_request(Request("What is USD/INR and then compute 12 * 9?")))
print(await handle_request(Request("plan: Draft a 2-step plan to compare two RBI circulars.")))




> Entering new AgentExecutor chain...

Invoking: `fx_rate_tool` with `{'pair': 'USD/INR'}`



Invoking: `calc_tool` with `{'expr': '12 * 9'}`


USD/INR ≈ 84.15 (mock)108Final Answer: USD/INR ≈ 84.15 (mock), 108

> Finished chain.
Final Answer: USD/INR ≈ 84.15 (mock), 108
Planner route unavailable: 'Kernel' object has no attribute 'complete_text_async'



## 5) Next Steps
- Add **LangSmith** tracing & governance.
- Replace mocks with real BFSI systems (rates API, risk engines).
- Combine with **LangGraph** for multi‑step flows and decision routing.
- Package with **FastAPI** and deploy (Azure/AWS).
